[Tutorial by Jurgen Hubner][def]

[def]: https://www.powerelectronicsnews.com/the-buck-regulator-power-supply-design-tutorial-part-2-1/

In [55]:
# Design parameters
VinMax = 47
VinNom = 45
VinMin = 40
Vo = 12
Vd = 0.5 # Schottky diode forward voltage drop (non-synchronous buck converter)
Δil_pp = 0.3 # A, 20% of 1A
fsw = 1.1e6 # Hz
Io_min = 0.15 # A
Io = 0.5 # A
Io_max = 1 # A
Lin = 1e-6 # H, input filter inductor, set to 1µH when not specified
η = 0.8 # Efficiency
ΔIo = 0.2 # A, load transient
ΔVo_load = 0.05 # V, load transient
Vref = 0.8 # V, reference voltage

# Duty cycle equations

In [56]:
Dmin = (Vo + Vd)/(VinMax + Vd)
Dnom = (Vo + Vd)/(VinNom + Vd)
Dmax = (Vo + Vd)/(VinMin + Vd)

# Inductor selection
1. Select the maximum peak to peak current ripple in the inductor: Δil_pp.  This is typically 20% to 40% of the maximum output current.

In [57]:
Lmin1 = Dmax*(VinMax - Vo)/(Δil_pp * fsw)

2. Guarantee CCM operation:

In [58]:
Lmin2 = Dmin*(VinMax - Vo)/(2* Io_min * fsw)

L1 = max(Lmin1, Lmin2)

# Input capacitor selection
1. Select the maximum input voltage ripple ΔVin_pp.  This is typically 1% to 5% of the minimum input voltage.
2. Calculate the minimum input capacitance required to maintain the input voltage ripple within the specified limits.

In [59]:

ΔVin_pp = VinMin * 0.01 # V
Cin_min = Io * (1-Dnom) * Dnom / (ΔVin_pp * fsw)

Filter Impedance and Converter Impedance

In [60]:
import math
# Filter impedance formed by input inductor and input capacitor

Zs = math.sqrt(Lin/Cin_min)

# Input impedance of the converter
Zin = -VinMin**2 / Vo * Io_max

if(abs(Zs) > abs(Zin)):
    print("The input filter impedance is too high")

If Zs is not much smaller than Zin, you'll have to add another capacitor (Cin2) with a high ESR.  This can be an electrolytic capacitor or an MLCC with a discrete series resistor.

In [61]:
Cin1 = Cin_min
Cin2 = 4* Cin1
ESRmin = 0.5 * math.sqrt(Lin/Cin1)

Zs = math.sqrt(Lin/(Cin1+Cin2))

# Damping factor, should be greater than 1

# Input capacitor RMS current calculation

In [62]:
I_source_max = Vo * Io_max / (η * VinMin)
Dmax = Vo / (VinMin * η)
Iin_avg = I_source_max / Dmax # Average height of the current peaks in the input current waveform

ΔiL = (VinMin - Vo) / (fsw * L1) * Dmax
Iin_pk = Iin_avg + ΔiL/2

# Input RMS current of the power stage
Iin_rms = math.sqrt(Dmax * (Iin_pk**2 + (ΔiL**2)/3 - Iin_pk * ΔiL))

# Input RMS current of input capacitors
Icin_rms = math.sqrt(Iin_rms**2 - I_source_max**2)

Group your input capacitors per type : MLCC, electrolytic, etc.
* MLCC ≤ 1µF can be skipped

Then calculate the RMS current for each group:

In [63]:
# Group 1
Xcin1 = 1/ (8*fsw * Cin1)
Zcin1 = math.sqrt(ESRmin**2 + Xcin1**2)

# Group 2
Xcin2 = 1/ (8*fsw * Cin2)
Zcin2 = math.sqrt(ESRmin**2 + Xcin2**2)

Ic1_rms = Icin_rms * Zcin2 / (Zcin1 + Zcin2)
Ic2_rms = Icin_rms * Zcin1 / (Zcin1 + Zcin2)

# Output capacitor selection

In [64]:
ΔVo_max_pp = 0.01 * Vo # V, up to 5% of Vo

# Equation set 1 with fudge factors for steady state
Co_min1 = Δil_pp / (4 * fsw * ΔVo_max_pp)
ESR_max1 = ΔVo_max_pp / (2 * Δil_pp)

# Equation set 2 with fudge factors for load transients
Co_min2 = 2 * ΔIo / (fsw * ΔVo_load)
ESR_max2 = ΔVo_load / (2 * ΔIo)

Co = max(Co_min1, Co_min2)
ESR_Co = min(ESR_max1, ESR_max2)

Group your output capacitors per type : MLCC, electrolytic, etc.
Calculate the peak-to-peak output voltage ripple ΔVo_pp from the load current ripple : ΔVo_pp = Δil_pp / ((1/Zo_bulk) + (1/Zo_ESR))

Then calculate the RMS current for each group:
Ico_rms = Δil_pp / sqrt(12)

# Feedback network design
After the resistor ratio is determined, the resistor values can be found [online](http://www.stratoloft.com/teledesign/ResistorRatio.html).

In [67]:
# Feedback resistor ratio : Rfbt/Rfbb
rr = (Vo-Vref)/Vref
Rfbb = 3.3e3 # Ohm
Rfbt = (Vo-Vref)/Vref * Rfbb # ohm, should be in the range of 10k to 100k

In [66]:
# LMR16030 
## UVLO calculation
# Rent = EN pull-up resistor
# Renb = EN pull-down resistor

Vstart = 40 # [V] UVLO rising threshold
Vstop = 35 # [V] UVLO  falling threshold 
Ihys = 3.6e-6 # [A] UVLO hysteresis curr$ent
Ven = 1.2 # [V] UVLO enable threshold
Ien = 1e-6 # [A] UVLO enable current
Rent = (Vstart - Vstop) / Ihys
Renb = Ven / ((Vstart-Ven)/Rent + Ien)

# Now fill in the real resistor values and calculate the real thresholds
Rent_real = 1.3e6 # [Ohm] EN pull-up resistor
Renb_real = 39e3 # [Ohm] EN pull-down resistor
Vstart_calc = (Ven / Renb_real - Ien) * Rent_real + Ven
Vstop_calc = Vstart_calc - Rent_real * Ihys